In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature=0.1
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)

examples = [
    {
        "movie": "탑건",
        "answer": """
        🛩️👨‍✈️🔥
        """,
    },
    {
        "movie": "대부",
        "answer": """
        👨‍👨‍👦🔫🍝
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 영화에 관한 엄청난 전문가야. 모든 영화에 대한것을 3개의 이모티콘으로 줄여서 보여줄 수 있어"),
        example_prompt,
        ("human", "What do you know about {movie}?"),
    ]
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result)

invoke_chain("더 코어")
invoke_chain("The day after Tomorrow")
invoke_chain("2012")



In [ ]:
prompt_question = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


chainRemember = RunnablePassthrough.assign(history=load_memory) | prompt_question | llm

def invoke_chain_about_question(question):
    result = chainRemember.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

invoke_chain_about_question("처음 물어봤던 영화는 뭐였어?")